In [11]:
import pandas as pd
import tqdm
from Bio import SeqIO
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# import keras functions
# import tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Lambda, Embedding
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import numpy as np
from Bio import SeqIO
from numpy import array
from numpy import argmax
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.backend import expand_dims
import matplotlib.pyplot as plt
from keras.regularizers import l1, l2


from keras.models import Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate


from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


In [12]:
def plot(history):
    # learning curves of model accuracy
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.show()

In [13]:
def get_input_for_embedding(fasta_file):
    encodings = []
    
    # define universe of possible input values
    alphabet = 'ARNDCQEGHILKMFPSTWYV-'
    
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    
    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        data = seq_record.seq
        for char in data:
            if char not in alphabet:
                return
        integer_encoded = [char_to_int[char] for char in data]
        encodings.append(integer_encoded)
    encodings = np.array(encodings)
    return encodings

In [14]:
# convert sequences to integer encoding, for embedding
#test_positive_embedding = get_input_for_embedding('data/DeepSuccinylSite/fasta/test_positive_sites.fasta')
#test_negative_embedding = get_input_for_embedding('data/DeepSuccinylSite/fasta/test_negative_sites.fasta')
#train_positive_embedding = get_input_for_embedding('data/DeepSuccinylSite/fasta/positive_sites.fasta')
#train_negative_embedding = get_input_for_embedding('data/DeepSuccinylSite/fasta/negative_sites.fasta')

# convert sequences to integer encoding, for embedding
test_positive_embedding = get_input_for_embedding('../data/test/fasta/test_positive_sites.fasta')
test_negative_embedding = get_input_for_embedding('../data/test/fasta/test_negative_sites.fasta')
train_positive_embedding = get_input_for_embedding('../data/train/fasta/positive_sites.fasta')
train_negative_embedding = get_input_for_embedding('../data/train/fasta/negative_sites.fasta')

# create labels
train_positive_labels = np.ones(train_positive_embedding.shape[0])
train_negative_labels = np.zeros(train_negative_embedding.shape[0])
test_positive_labels = np.ones(test_positive_embedding.shape[0])
test_negative_labels = np.zeros(test_negative_embedding.shape[0])

# stack positive and negative data together
X_train_full_embedding = np.vstack((train_positive_embedding,train_negative_embedding))
X_test_embedding = np.vstack((test_positive_embedding,test_negative_embedding))
y_train_full = np.concatenate((train_positive_labels, train_negative_labels), axis = 0)
y_test = np.concatenate((test_positive_labels, test_negative_labels), axis = 0)

#train_positive_pt5 = pd.read_csv("data/DeepSuccinylSite/features/full/train_positive_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]
#train_negative_pt5 = pd.read_csv("data/DeepSuccinylSite/features/full/train_negative_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]
#test_positive_pt5 = pd.read_csv("data/DeepSuccinylSite/features/full/test_positive_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]
#test_negative_pt5 = pd.read_csv("data/DeepSuccinylSite/features/full/test_negative_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]

train_positive_pt5 = pd.read_csv("../data/train/features/train_positive_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]
train_negative_pt5 = pd.read_csv("../data/train/features/train_negative_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]
test_positive_pt5 = pd.read_csv("../data/test/features/test_positive_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]
test_negative_pt5 = pd.read_csv("../data/test/features/test_negative_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]


# stack positive and negative data together
X_train_pt5_full = np.vstack((train_positive_pt5,train_negative_pt5))
X_test_pt5 = np.vstack((test_positive_pt5,test_negative_pt5))


In [15]:
# shuffle X and y together
# X_train_pt5_full, X_train_full_embedding, y_train = shuffle(X_train_pt5_full, X_train_full_embedding, y_train_full)
# X_test_pt5, X_test_embedding, y_test = shuffle(X_test_pt5, X_test_embedding, y_test)

In [16]:
# model.summary()
# plot_model(model, show_shapes=True)

In [17]:
'''def CNN_Embedding():
    # Embedding
    model = Sequential()
    model.add(Embedding(256, 21, input_length=33))
    model.add(Lambda(lambda x: K.expand_dims(x, 3)))
    model.add(Conv2D(32, kernel_size=(17, 3), activation = 'relu', kernel_initializer='he_normal', padding = 'VALID'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(16, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model'''

#import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Layer, GlobalAveragePooling1D, Input, GlobalAveragePooling2D
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.models import Sequential


class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=rate)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        
    def get_config(self):
        config = super().get_config()
        return config

    def call(self, inputs, training):
        attn_output, attention_scores = self.att(inputs, inputs, inputs, return_attention_scores=True, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output), attention_scores


class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim, mask_zero=True)
        
    def get_config(self):
        config = super().get_config()
        return config

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

def embedding_model():
    # transformer based embedding model
    vocab_size = 21
    embed_dim = 128 #config["embedding_dim"]
    ff_dim = 128 #config["feed_forward_dim"]
    max_len = 33 #config["maximum_path_length"]
    dropout = 0.2 #config["dropout"]
    n_heads = 8

    inputs = Input(shape=(max_len,))
    embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, n_heads, ff_dim)
    x, weights = transformer_block(x)
    print(x.shape, weights.shape)
    x = GlobalAveragePooling1D()(x) #GlobalAveragePooling1D
    x = Dropout(dropout)(x)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(16, activation="relu")(x)
    x = Dropout(dropout)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    return Model(inputs=inputs, outputs=[outputs])

### Iterated 10-Fold CROSS VALIDATION

In [19]:
n_k = 10
kfold = KFold(n_k, shuffle=True)

n_epo = 100
batch_size = 32
# initialize average variables
avg_acc, avg_mcc, avg_sp, avg_sn = 0, 0, 0, 0

for train, val in kfold.split(X_train_full_embedding, y_train_full):
    
    # Early stopping
    es = EarlyStopping(monitor='val_accuracy', patience=7, mode='auto')

    # Checkpointer
    metric = 'val_accuracy'
    checkpointer = ModelCheckpoint(filepath="models/transformer_st_model_best.h5",
                            monitor = metric,
                            verbose=0, 
                            save_weights_only=False,
                            save_best_only=True)

    model = embedding_model() #CNN_Embedding()
    model.compile(optimizer=Adam(learning_rate=1e-3),
              loss=BinaryCrossentropy(),
              metrics=['accuracy'])
    model.summary()
    # seperate val data
    X_train, X_val = X_train_full_embedding[train], X_train_full_embedding[val]
    y_train, y_val = y_train_full[train], y_train_full[val]

    # Training and Evaluation
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epo, verbose=1, callbacks=[checkpointer],
                            validation_data=(X_val, y_val))


    y_pred = model.predict(X_val).reshape(y_val.shape[0],)

    y_pred = (y_pred > 0.5)
    y_pred = [int(i) for i in y_pred]
    y_val = np.array(y_val)
    y_pred = np.array(y_pred)

    cm = confusion_matrix(y_val, y_pred)
    mcc = matthews_corrcoef(y_val, y_pred)
    acc = accuracy_score(y_val, y_pred)
    sn = cm[1][1]/(cm[1][1]+cm[1][0])
    sp = cm[0][0]/(cm[0][0]+cm[0][1])
    print(acc,mcc,sn,sp)
    # plot(history)

(None, 33, 128) (None, 8, 33, 33)
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 33)]              0         
                                                                 
 token_and_position_embeddin  (None, 33, 128)          6912      
 g_6 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block_6 (Transf  ((None, 33, 128),        561024    
 ormerBlock)                  (None, 8, 33, 33))                 
                                                                 
 global_average_pooling1d_6   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_32 (Dropout)    

KeyboardInterrupt: 

In [ ]:
#import tensorflow as tf
gpu = tf.config.list_physical_devices('GPU')
if len(gpu) > 0:
    from numba import cuda
    cuda.select_device(0)
    cuda.close()